This notebook focuses on finetuning the DistilBERT model from Hugging Face to classify emotions in text.

I used the Go emotions dataset as its multilabel aspect is ideal for journal entires which often reflect a myriad of emtions.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#installing the necessary libraries
!pip install -q transformers datasets scikit-learn

#Import the necessary dependancies
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer #tokenizer & model, training args and trainer
from datasets import load_dataset, DatasetDict  #accessing the dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

#Load dataset
ds = load_dataset("google-research-datasets/go_emotions", "simplified")


#Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=27)

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
#This ensures emotion_labels is available globally in your notebook/session before you use it anywhere
labels_feature = ds["train"].features["labels"]

# If it's a ClassLabel or Sequence
if hasattr(labels_feature, "feature"):
    emotion_labels = labels_feature.feature.names  # For Sequence feature of ClassLabel
else:
    emotion_labels = labels_feature.names  # For ClassLabel itself

print(emotion_labels)



['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:

#Tokenizing the dataset:to ensure consistent input format to model
# The tokenizer converts raw text into token IDs
# Padding ensures all sequences are the same length
# Truncation cuts off very long sequences so they fit the model
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

# Apply the tokenizer across the entire dataset
# batched=True = process multiple examples at once for speed
tokenized_dataset = ds.map(tokenize, batched=True)

# Check one sample after tokenization
print(tokenized_dataset["train"][0])


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej', 'input_ids': [101, 2026, 8837, 2833, 2003, 2505, 1045, 2134, 1005, 1056, 2031, 2000, 5660, 2870, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:

import numpy as np
THRESHOLD = 0.2

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # model output vs. true labels
    logits = np.asarray(logits)
    labels = np.asarray(labels)

    # Convert logits -> probabilities unless already [0,1]
    if (logits.min() >= 0.0) and (logits.max() <= 1.0):
        probs = logits
    else:
        probs = sigmoid(logits)

    preds = (probs >= THRESHOLD).astype(int)

    return {
        # Micro: aggregates TP/FP/FN across all labels
        "precision_micro": precision_score(labels, preds, average="micro",  zero_division=0),
        "recall_micro":    recall_score(labels,  preds, average="micro",    zero_division=0),
        "f1_micro":        f1_score(labels,      preds, average="micro",    zero_division=0),

        # Macro: unweighted mean over labels (reveals rare-class weakness)
        "precision_macro": precision_score(labels, preds, average="macro",  zero_division=0),
        "recall_macro":    recall_score(labels,  preds, average="macro",    zero_division=0),
        "f1_macro":        f1_score(labels,      preds, average="macro",    zero_division=0),

        # Optional extras you can keep or drop:
        # "f1_samples":      f1_score(labels,      preds, average="samples",  zero_division=0),
        # "subset_accuracy": accuracy_score(labels, preds),
    }


In [4]:

# Assumes: `ds` (datasets.DatasetDict) and `tokenizer` are already in the notebook workspace.

# Imports for loading models
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import Sequence, Value
import numpy as np
import torch


# (2) constants
NUM_LABELS = 27
MAX_LEN = 128

# (3) tokenizer mapping (tokenize texts)
def tokenize_fn(batch):#loading a batch of examples to BERT tokenizer
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

# Dataset mapping for multilabel flattening
def to_multilabel_flat(batch):

    out_labels = []
    for labs in batch["labels"]:
        # handle numpy / torch / nested single-element wrappers
        if isinstance(labs, (np.ndarray, torch.Tensor)):
            labs = labs.tolist()

        # if nested wrapper like [[...]] -> unwrap
        if isinstance(labs, list) and len(labs) == 1 and isinstance(labs[0], (list, np.ndarray, torch.Tensor)):
            labs = labs[0]
            if isinstance(labs, (np.ndarray, torch.Tensor)):
                labs = labs.tolist()

        # Case 1: list of integer indices -> build multi-hot vector
        if isinstance(labs, list) and all(isinstance(x, int) for x in labs):
            row = [0.0] * NUM_LABELS
            for idx in labs:
                if 0 <= idx < NUM_LABELS:
                    row[idx] = 1.0
            out_labels.append(row)
            continue #Incase theyre integers, Builds a one-hot vector of length 27, setting 1.0 for each emotion index present.

        # Case 2: already a sequence of 0/1 values or floats (maybe wrong length) -> normalize
        if isinstance(labs, list) and all(isinstance(x, (int, float)) for x in labs):
            # convert all to float
            arr = [float(x) for x in labs]
            # pad or truncate to NUM_LABELS
            if len(arr) < NUM_LABELS:
                arr = arr + [1.0 * 0.0] * (NUM_LABELS - len(arr))
            elif len(arr) > NUM_LABELS:
                arr = arr[:NUM_LABELS]
            out_labels.append(arr)
            continue

        # Fallback: unexpected shape/types -> create zero vector (safe)
        out_labels.append([0.0] * NUM_LABELS)

    batch["labels"] = out_labels
    return batch

tokenized = ds.map(tokenize_fn, batched=True)
tokenized = tokenized.map(to_multilabel_flat, batched=True)

# Force the Arrow schema of labels to float32 vectors of fixed length
for split in tokenized.keys():
    tokenized[split] = tokenized[split].cast_column(
        "labels", Sequence(Value("float32"), length=NUM_LABELS)
    )#to prevent datatype issues when loading to pytorch

# (C) finally set HF dataset format so Trainer gets tensors
# Do NOT pass dtype as dict (that caused earlier TypeError). If labels are floats (Python lists), HF will convert to torch.float.
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Sanity checks (confirm require keys, that all are torch.Tensor,Label shape, datatype etc...)
example = tokenized["train"][0]
print("keys:", example.keys())
print("input_ids dtype:", example["input_ids"].dtype, "shape:", example["input_ids"].shape)#input is in a 'vocabulary' model understands
print("attention_mask dtype:", example["attention_mask"].dtype, "shape:", example["attention_mask"].shape)#tells model what to pay attention to
print("labels dtype:", example["labels"].dtype, "shape:", example["labels"].shape)  # should be torch.float32 and (27,) emotions
print("labels sample:", example["labels"])

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/43410 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5426 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5427 [00:00<?, ? examples/s]

keys: dict_keys(['labels', 'input_ids', 'attention_mask'])
input_ids dtype: torch.int64 shape: torch.Size([128])
attention_mask dtype: torch.int64 shape: torch.Size([128])
labels dtype: torch.float32 shape: torch.Size([27])
labels sample: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])


In [5]:
import transformers, inspect
print("transformers version:", transformers.__version__)
from transformers import TrainingArguments
print("TrainingArguments class:", TrainingArguments)
print("Init signature:", inspect.signature(TrainingArguments.__init__))


transformers version: 4.57.1
TrainingArguments class: <class 'transformers.training_args.TrainingArguments'>
Init signature: (self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, 

In [6]:
ex = tokenized["train"][0]
assert ex["labels"].dtype == torch.float32
assert ex["labels"].shape[-1] == NUM_LABELS


In [10]:
#first train run
# --- Hard-disable W&B ---
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

# --- Imports ---
import numpy as np
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import DataCollatorWithPadding
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

# Assumes you already have: tokenizer, tokenized (DatasetDict), NUM_LABELS, MAX_LEN


train_subset = tokenized["train"].shuffle(seed=42).select(range(min(5000, len(tokenized["train"]))))
eval_subset  = tokenized["validation"].shuffle(seed=42).select(range(min(1000, len(tokenized["validation"]))))

# --- Model (multi-label) ---
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=NUM_LABELS,
)
model.config.problem_type = "multi_label_classification"
model.config.num_labels   = NUM_LABELS

# --- Collator that forces labels -> float32 (so BCEWithLogitsLoss works) ---
base_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def float_label_collator(features):
    # Separate labels first (they may be int64 in the dataset)
    labels = [f["labels"] for f in features]
    # Use base collator to pad input_ids / attention_mask
    batch = base_collator([{k: v for k, v in f.items() if k != "labels"} for f in features])
    # Cast labels to float32 and stack
    batch["labels"] = torch.tensor(labels, dtype=torch.float32)
    return batch

# --- Metrics (scikit-learn) ---
THRESHOLD = 0.2  # tune later or per-class

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = np.asarray(logits); labels = np.asarray(labels)
    # logits -> probabilities via sigmoid (unless already in [0,1])
    if (logits.min() >= 0.0) and (logits.max() <= 1.0):
        probs = logits
    else:#uses sigmoid to squash each output to [0,1]= probability the emotion is present
        probs = 1.0 / (1.0 + np.exp(-logits))
    preds = (probs >= THRESHOLD).astype(int)#only predict for those above the set threshold

    return {
        "precision_micro": precision_score(labels, preds, average="micro",  zero_division=0),
        "recall_micro":    recall_score(labels,  preds, average="micro",    zero_division=0),
        "f1_micro":        f1_score(labels,      preds, average="micro",    zero_division=0),
        "precision_macro": precision_score(labels, preds, average="macro",  zero_division=0),
        "recall_macro":    recall_score(labels,  preds, average="macro",    zero_division=0),
        "f1_macro":        f1_score(labels,      preds, average="macro",    zero_division=0),
        "f1_samples":      f1_score(labels,      preds, average="samples",  zero_division=0),
        "subset_accuracy": accuracy_score(labels, preds),
    }

# --- TrainingArguments (HF 4.56.2 uses eval_strategy) ---
training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    logging_steps=50,
    report_to="none",            # no wandb/tensorboard
    run_name=None,
    dataloader_pin_memory=False, # silence CPU-only warning
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,   # <-- critical: casts labels to float32
)

# --- Train & Evaluate ---
trainer.train()
eval_metrics = trainer.evaluate()
print(eval_metrics)
model_save_path = "/content/drive/MyDrive/finetuned_distilbert_goemotions"


# --- Save ---
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Training finished. Model saved to {model_save_path}")



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro,Precision Macro,Recall Macro,F1 Macro,F1 Samples,Subset Accuracy
1,0.135600,0.134591,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.292000
2,0.127100,0.124521,0.612903,0.066279,0.119622,0.041026,0.037274,0.033142,0.047900,0.326000
3,0.118100,0.117889,0.563636,0.144186,0.229630,0.104935,0.070796,0.067372,0.110567,0.366000


{'eval_loss': 0.11788921803236008, 'eval_precision_micro': 0.5636363636363636, 'eval_recall_micro': 0.14418604651162792, 'eval_f1_micro': 0.22962962962962963, 'eval_precision_macro': 0.1049352651567081, 'eval_recall_macro': 0.07079610942957047, 'eval_f1_macro': 0.06737214995308231, 'eval_f1_samples': 0.11056666666666666, 'eval_subset_accuracy': 0.366, 'eval_runtime': 3.6223, 'eval_samples_per_second': 276.065, 'eval_steps_per_second': 8.834, 'epoch': 3.0}
Training finished. Model saved to /content/drive/MyDrive/finetuned_distilbert_goemotions


In [11]:
#checking the order of emotin label list
from datasets import load_dataset

# Load dataset (the "simplified" config is what you're using)
ds = load_dataset("go_emotions", "simplified")
print(ds["train"].features["labels"].feature.names)


['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [13]:
#first testcase with 0.2 global threshold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
    'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
    'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness',
    'surprise', 'neutral'
]

# Set the actual model path
model_path = "/content/drive/MyDrive/finetuned_distilbert_goemotions"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example input
text = "I was irresponsible with money and mum is upset with me."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

# Inference
with torch.no_grad():
    logits = model(**inputs).logits.numpy()[0]

# Apply sigmoid and threshold
THRESHOLD = 0.2
probs = 1.0 / (1.0 + np.exp(-logits))
preds = (probs >= THRESHOLD).astype(int)

# Print the predicted emotions by name instead of vector
pred_emotions = [emotion_labels[i] for i, v in enumerate(preds) if v == 1]
print("Predicted emotions:", pred_emotions)


Predicted emotions: []


* Tests case shows no label for predicted
emotion as recall and f1 score are too low, model is missing a lot of emotions.

* The global threshold is also too high hence no emotions ouput
I'll attempt to improve the models performance by increasing the number of training epochs, lowering the learning rate, lowering the threshold and applying per-class thresholding.
* Ill also use a larger subset for training

In [14]:
#these are diagnostic attempts to try figure out issues
#so far suspected issues:
#1)Imbalanced labels (many negatives per class) → model biases to predict negative (0) to avoid false positives.
#2)Run diagnostics (class counts & probability distributions).
#3)If diagnostics show extreme imbalance or very low pos_means, compute per-class thresholds and re-evaluate; note the new micro-F1.
#4)If still low, pick either pos_weight or oversampling and run a retrain with learning_rate=1e-5, epochs=4


import numpy as np

# tokenized/validation assumed available and tokenizer/model loaded
val = tokenized["validation"]
labels = np.stack([ex["labels"] for ex in val])  # shape (N, NUM_LABELS)
pos_counts = labels.sum(axis=0).astype(int)
print("pos_counts per class:", pos_counts)
print("total samples:", labels.shape[0])
print("label density (avg positives per sample):", labels.sum() / labels.shape[0])


pos_counts per class: [488 303 195 303 397 153 152 248  77 163 292  97  35  96  90 358  13 172
 252  21 209  15 127  18  68 143 129]
total samples: 5426
label density (avg positives per sample): 0.85035014


In [15]:
# --- One-block diagnostic: recreate float_label_collator + compute probs/stats ---
from torch.utils.data import DataLoader
import torch
import numpy as np

# Recreate the float_label_collator used in training (exact behavior)
from transformers import DataCollatorWithPadding
base_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def float_label_collator(features):
    labels = [f["labels"] for f in features]
    batch = base_collator([{k: v for k, v in f.items() if k != "labels"} for f in features])
    batch["labels"] = torch.tensor(np.array(labels), dtype=torch.float32)

    return batch

# Prepare dataloader for validation
val = tokenized["validation"]                # assume this exists
device = model.device if hasattr(model, "device") else ("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

dl = DataLoader(val, batch_size=64, collate_fn=float_label_collator)

all_probs = []
all_labels = []
with torch.no_grad():
    for b in dl:
        inputs = {k: v.to(device) for k, v in b.items() if k != "labels"}
        logits = model(**inputs).logits
        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)
        all_labels.append(b["labels"].numpy())

all_probs = np.vstack(all_probs)
all_labels = np.vstack(all_labels)

# Summary stats
print("probs min,mean,max:", all_probs.min(), all_probs.mean(), all_probs.max())
print("shape probs:", all_probs.shape, "shape labels:", all_labels.shape)
print("avg positives per sample in GT:", all_labels.sum()/all_labels.shape[0])

# Per-class diagnostics
for i in range(all_probs.shape[1]):
    pos_mask = all_labels[:, i] == 1
    if pos_mask.sum() > 0:
        pos_mean = all_probs[pos_mask, i].mean()
        neg_mean = all_probs[~pos_mask, i].mean()
        print(f"class {i}: pos_mean={pos_mean:.4f}, neg_mean={neg_mean:.4f}, pos_count={pos_mask.sum()}")
    else:
        print(f"class {i}: no positives")


probs min,mean,max: 0.0047914027 0.035838723 0.69454104
shape probs: (5426, 27) shape labels: (5426, 27)
avg positives per sample in GT: 0.85035014
class 0: pos_mean=0.2234, neg_mean=0.0829, pos_count=488
class 1: pos_mean=0.0530, neg_mean=0.0496, pos_count=303
class 2: pos_mean=0.0582, neg_mean=0.0372, pos_count=195
class 3: pos_mean=0.0828, neg_mean=0.0581, pos_count=303
class 4: pos_mean=0.0707, neg_mean=0.0675, pos_count=397
class 5: pos_mean=0.0308, neg_mean=0.0304, pos_count=153
class 6: pos_mean=0.0490, neg_mean=0.0335, pos_count=152
class 7: pos_mean=0.1477, neg_mean=0.0445, pos_count=248
class 8: pos_mean=0.0279, neg_mean=0.0241, pos_count=77
class 9: pos_mean=0.0401, neg_mean=0.0365, pos_count=163
class 10: pos_mean=0.0590, neg_mean=0.0465, pos_count=292
class 11: pos_mean=0.0333, neg_mean=0.0264, pos_count=97
class 12: pos_mean=0.0128, neg_mean=0.0124, pos_count=35
class 13: pos_mean=0.0316, neg_mean=0.0266, pos_count=96
class 14: pos_mean=0.0188, neg_mean=0.0175, pos_count=

In [ ]:
#pos_mean ≈ neg_mean for many classes, the model hasn’t separated them well

In [17]:
#finetuning for better metrics with a lower, per-class, threshold for higher sensitivity, higher epochs, more powerful processor(T4-GPU)
# lower learning rate
#Since probabilities cluster around 0.5, adjust decision thresholds per class
#Pos_weight in BCEWithLogitsLoss: gives higher penalty for false negatives on rare classes.
#Implementing these will improve gradient signals and force the model to better separate classes.
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import (
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

# --- STEP 1: Calculate pos_weight for each class from training data ---
# pos_weight[i] = num_negatives / num_positives for class i
# This penalizes false negatives more heavily for rare classes

def calculate_pos_weight(dataset, num_labels):
    """Calculate pos_weight tensor for BCEWithLogitsLoss"""
    labels_array = np.array([example["labels"] for example in dataset])
    pos_counts = labels_array.sum(axis=0)  # positives per class
    neg_counts = len(labels_array) - pos_counts  # negatives per class

    # pos_weight = neg_count / pos_count
    pos_weight = neg_counts / np.maximum(pos_counts, 1)  # avoid division by zero

    print("=== Class Distribution & pos_weight ===")
    for i in range(num_labels):
        print(f"Class {i}: pos={int(pos_counts[i])}, neg={int(neg_counts[i])}, pos_weight={pos_weight[i]:.4f}")

    return torch.tensor(pos_weight, dtype=torch.float32)


# --- ASSUMES YOU HAVE: tokenizer, tokenized (DatasetDict), NUM_LABELS, MAX_LEN ---

# Use a larger data sample for training/eval
train_subset = tokenized["train"]  # remove shuffle/select to use full dataset
eval_subset  = tokenized["validation"]
pos_weight = calculate_pos_weight(train_subset, NUM_LABELS)


# --- STEP 2: Custom Trainer with pos_weight ---
class WeightedBCETrainer(Trainer):
    def __init__(self, pos_weight=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Move pos_weight to same device as logits
        pos_weight_device = self.pos_weight.to(logits.device) if self.pos_weight is not None else None

        # BCEWithLogitsLoss with pos_weight
        loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weight_device)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


# --- Model ---
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=NUM_LABELS,
)
model.config.problem_type = "multi_label_classification"
model.config.num_labels   = NUM_LABELS

# --- Collator: force labels to float32 ---
base_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def float_label_collator(features):
    labels = [f["labels"] for f in features]
    batch = base_collator([{k: v for k, v in f.items() if k != "labels"} for f in features])
    #explicitly stacks tensors of equal shape along a new dimension
    batch["labels"] = torch.stack(labels).float()
    return batch

#train with default threshold of 0.5, ill optimize after
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1.0 / (1.0 + np.exp(-np.asarray(logits))) #sigmoid fnctn
    preds = (probs >= 0.5).astype(int)

    return {
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro":    recall_score(labels,  preds, average="micro", zero_division=0),
        "f1_micro":        f1_score(labels,      preds, average="micro", zero_division=0),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro":    recall_score(labels,  preds, average="macro", zero_division=0),
        "f1_macro":        f1_score(labels,      preds, average="macro", zero_division=0),
        "f1_samples":      f1_score(labels,      preds, average="samples", zero_division=0),
        "subset_accuracy": accuracy_score(labels, preds),
    }

# --- TrainingArguments ---
training_args = TrainingArguments(
    output_dir="/outputs_run1",                       #checkpoint
    per_device_train_batch_size=16,              # can be tuned
    per_device_eval_batch_size=32,
    learning_rate=1e-5,                          # lower for stability, tune if training is too slow or plateauing
    num_train_epochs=6,                         # increase for more learning
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    logging_steps=50,
    report_to="none",
    run_name=None,
    dataloader_pin_memory=False,

)

# --- Initialize WeightedBCETrainer with pos_weight---
trainer = WeightedBCETrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    data_collator=float_label_collator,
    pos_weight=pos_weight,
)

# --- Train & Evaluate ---
trainer.train()
# --- STEP 9: Optimize Per-Class Thresholds on Validation Set ---
print("\n=== Optimizing Per-Class Thresholds ===")

# Get predictions on validation set
eval_predictions = trainer.predict(eval_subset)
logits = eval_predictions.predictions
labels = eval_predictions.label_ids
probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid

# Find optimal threshold for each class (maximize F1)
optimal_thresholds = []
for class_idx in range(NUM_LABELS):
    best_f1 = 0
    best_threshold = 0.5

    # Try thresholds from 0.1 to 0.9
    for threshold in np.arange(0.1, 0.91, 0.05):
        preds = (probs[:, class_idx] >= threshold).astype(int)
        f1 = f1_score(labels[:, class_idx], preds, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    optimal_thresholds.append(best_threshold)
    print(f"Class {class_idx}: optimal_threshold={best_threshold:.2f}, F1={best_f1:.4f}")

optimal_thresholds = np.array(optimal_thresholds)


# --- STEP 10: Evaluate with Optimized Thresholds ---
print("\n=== Final Evaluation with Optimized Thresholds ===")

preds_optimized = np.zeros_like(probs, dtype=int)
for i, threshold in enumerate(optimal_thresholds):
    preds_optimized[:, i] = (probs[:, i] >= threshold).astype(int)

final_metrics = {
    "precision_micro": precision_score(labels, preds_optimized, average="micro", zero_division=0),
    "recall_micro": recall_score(labels, preds_optimized, average="micro", zero_division=0),
    "f1_micro": f1_score(labels, preds_optimized, average="micro", zero_division=0),
    "precision_macro": precision_score(labels, preds_optimized, average="macro", zero_division=0),
    "recall_macro": recall_score(labels, preds_optimized, average="macro", zero_division=0),
    "f1_macro": f1_score(labels, preds_optimized, average="macro", zero_division=0),
    "f1_samples": f1_score(labels, preds_optimized, average="samples", zero_division=0),
    "subset_accuracy": accuracy_score(labels, preds_optimized),
}

print("\nFinal Metrics with Optimized Thresholds:")
for key, value in final_metrics.items():
    print(f"  {key}: {value:.4f}")


# --- STEP 11: Save Model and Thresholds ---
model_save_path = "/content/drive/MyDrive/finetuned_distilbert_goemotions"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Save optimal thresholds for inference
np.save(f"{model_save_path}/optimal_thresholds.npy", optimal_thresholds)
print(f"\n✓ Training finished. Model and thresholds saved to {model_save_path}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== Class Distribution & pos_weight ===
Class 0: pos=4130, neg=39280, pos_weight=9.5109
Class 1: pos=2328, neg=41082, pos_weight=17.6469
Class 2: pos=1567, neg=41843, pos_weight=26.7026
Class 3: pos=2470, neg=40940, pos_weight=16.5749
Class 4: pos=2939, neg=40471, pos_weight=13.7703
Class 5: pos=1087, neg=42323, pos_weight=38.9356
Class 6: pos=1368, neg=42042, pos_weight=30.7325
Class 7: pos=2191, neg=41219, pos_weight=18.8129
Class 8: pos=641, neg=42769, pos_weight=66.7223
Class 9: pos=1269, neg=42141, pos_weight=33.2080
Class 10: pos=2022, neg=41388, pos_weight=20.4688
Class 11: pos=793, neg=42617, pos_weight=53.7415
Class 12: pos=303, neg=43107, pos_weight=142.2673
Class 13: pos=853, neg=42557, pos_weight=49.8910
Class 14: pos=596, neg=42814, pos_weight=71.8356
Class 15: pos=2662, neg=40748, pos_weight=15.3073
Class 16: pos=77, neg=43333, pos_weight=562.7662
Class 17: pos=1452, neg=41958, pos_weight=28.8967
Class 18: pos=2086, neg=41324, pos_weight=19.8102
Class 19: pos=164, neg=432

Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro,Precision Macro,Recall Macro,F1 Macro,F1 Samples,Subset Accuracy
1,0.725600,0.740824,0.186504,0.839185,0.305182,0.216256,0.792248,0.312880,0.308678,0.075931
2,0.602100,0.658210,0.209831,0.853923,0.336882,0.236496,0.825335,0.343965,0.331077,0.085146
3,0.514000,0.636496,0.218398,0.852622,0.347726,0.235542,0.825007,0.346018,0.337445,0.083118
4,0.528300,0.673177,0.251670,0.824881,0.385672,0.262092,0.802158,0.375221,0.363055,0.133800
5,0.549200,0.693709,0.258657,0.824014,0.393724,0.266842,0.793208,0.380489,0.367283,0.138961
6,0.432800,0.709558,0.265345,0.820763,0.401038,0.270498,0.787749,0.385010,0.371637,0.149466



=== Optimizing Per-Class Thresholds ===


Class 0: optimal_threshold=0.85, F1=0.7149
Class 1: optimal_threshold=0.80, F1=0.7858
Class 2: optimal_threshold=0.85, F1=0.4699
Class 3: optimal_threshold=0.85, F1=0.3915
Class 4: optimal_threshold=0.70, F1=0.3781
Class 5: optimal_threshold=0.90, F1=0.5116
Class 6: optimal_threshold=0.90, F1=0.4104
Class 7: optimal_threshold=0.90, F1=0.5369
Class 8: optimal_threshold=0.90, F1=0.5333
Class 9: optimal_threshold=0.85, F1=0.3294
Class 10: optimal_threshold=0.90, F1=0.4255
Class 11: optimal_threshold=0.90, F1=0.4078
Class 12: optimal_threshold=0.85, F1=0.5319
Class 13: optimal_threshold=0.90, F1=0.3304
Class 14: optimal_threshold=0.90, F1=0.6154
Class 15: optimal_threshold=0.85, F1=0.9011
Class 16: optimal_threshold=0.85, F1=0.3333
Class 17: optimal_threshold=0.90, F1=0.5344
Class 18: optimal_threshold=0.90, F1=0.7743
Class 19: optimal_threshold=0.90, F1=0.2941
Class 20: optimal_threshold=0.90, F1=0.5966
Class 21: optimal_threshold=0.85, F1=0.7407
Class 22: optimal_threshold=0.90, F1=0.261

In [ ]:
#model performance summary with optimised threholds shows a clear improvemnt in its peformance
#balanced f1, precision and recall fairly well as can be seen on the test cases below

In [24]:
#TEST CASE 2
import torch
import numpy as np

# Assume 'model', 'tokenizer', 'THRESHOLDS', 'NUM_LABELS' already loaded from your fine-tuned state

def predict_emotions(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Move inputs to model's device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu().numpy()

    probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid to get probabilities
    preds = (probs >= threshold).astype(int)

    # Map to emotion labels (ensure emotion_labels list corresponds)
    emotion_labels = ds["train"].features["labels"].feature.names
    predicted_emotions = [emotion_labels[i] for i, pred in enumerate(preds[0]) if pred == 1]

    return predicted_emotions, probs[0]

# Test on your example text
example_text = "Im glad i started saving more and being more mindful of my expenses. its made my weekly schedule more manageable"
emotions, probabilities = predict_emotions(example_text)



print("Predicted emotions:", emotions)
print("Probabilities:", probabilities)


Predicted emotions: ['disappointment', 'sadness']
Probabilities: [0.07750715 0.02799528 0.06757093 0.3607183  0.2788744  0.33025634
 0.1761651  0.03106021 0.05315551 0.960114   0.48868656 0.30422968
 0.6176638  0.04192197 0.49891174 0.04698573 0.11731828 0.14156446
 0.0305586  0.82620347 0.14469215 0.04165073 0.6108349  0.38650197
 0.5530398  0.9620109  0.06595667]


In [27]:
# Test on your example text
example_text = "I was happy to see my friend today."
emotions, probabilities = predict_emotions(example_text)



print("Predicted emotions:", emotions)
print("Probabilities:", probabilities)

Predicted emotions: ['joy']
Probabilities: [0.21885096 0.19217704 0.00639465 0.02876747 0.19509463 0.38799763
 0.00931915 0.01667097 0.12848209 0.02615014 0.01618107 0.00437108
 0.00233195 0.7533829  0.0046423  0.39130256 0.03572585 0.99511176
 0.36165574 0.00949147 0.1961632  0.08887191 0.07403798 0.4655116
 0.00629101 0.0782409  0.01101147]


In [21]:
#downloading model for use
import shutil

model_path = "/content/drive/MyDrive/finetuned_distilbert_goemotions"
zip_path = "/content/model_for_download"

print(" Zipping model files...")
shutil.make_archive(zip_path, 'zip', model_path)

print("✅ Model zipped! Download it from Colab's file browser:")


 Zipping model files...
✅ Model zipped! Download it from Colab's file browser:
